This notebook is just to get some intution to eager execution which is a recent update in tensorflow.
Here I used a CIFAR dataset which can be downloaded from the internet

In [ ]:
#loading all the packages needed
import pickle
import numpy as np
import sys
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import tensorflow.contrib.eager as tce
tf.enable_eager_execution()

In [ ]:
def load_CIFAR10(filename):                              #used to extract dataset

  with open(filename, 'rb') as f:
    if sys.version_info[0] < 3:
      dict = pickle.load(f)
    else:
      dict = pickle.load(f, encoding='latin1')
    x = dict['data']
    y = dict['labels']
    x = x.astype(float)
    y = np.array(y)
  return x, y

In [ ]:
def load_data():       
 #used to extract the data. 

  xs = []
  ys = []
  for i in range(1, 6):
    filename = 'G:\\My_ml_work\\Basic_softmax\\cifar-10-python\\cifar-10-batches-py\\data_batch_' + str(i)
    X, Y = load_CIFAR10(filename)
    xs.append(X)
    ys.append(Y)
 

  x_train = np.concatenate(xs)
  y_train = np.concatenate(ys)
  del xs, ys
  x_test, y_test = load_CIFAR10_batch('G:\\My_ml_work\\Basic_softmax\\cifar-10-python\\cifar-10-batches-py\\test_batch')

  classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse',
    'ship', 'truck']

  # Normalize Data
  mean_image = np.mean(x_train, axis=0)
  x_train -= mean_image
  x_test -= mean_image

  data_dict = {
    'images_train': x_train,
    'labels_train': y_train,
    'images_test': x_test,
    'labels_test': y_test,
    'classes': classes
  }
  return data_dict


In [ ]:
batch_size = 100
learning_rate = 0.005

In [ ]:
data_sets = load_data()

Here I defined a simple softmax classifier.So,we can't expect higher accuracy from the model. 

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Dense(10, activation="linear",input_shape = (3072,))])

I defined two functions which are used to calculate loss and gradients

In [ ]:
def loss(model,x,y):
    y_ = model(x)
    return tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_,labels=y)

In [ ]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)  #here I used an adam optimizer

In [ ]:
num_epochs = 4000

for epoch in range(num_epochs):
    epoch_loss_avg = tce.metrics.Mean()
    epoch_accuracy = tce.metrics.Accuracy()
    indices = np.random.choice(data_sets['images_train'].shape[0], batch_size)
    images_batch = data_sets['images_train'][indices]
    labels_batch = data_sets['labels_train'][indices]
    #all the above steps are used to create batches
    grads = grad(model, images_batch, labels_batch)
    optimizer.apply_gradients(zip(grads, model.variables),
                              global_step=tf.train.get_or_create_global_step())
    # Track progress
    epoch_loss_avg(loss(model, images_batch, labels_batch))  # add current batch loss
    # compare predicted label to actual label
    epoch_accuracy(tf.argmax(model(images_batch), axis=1, output_type=tf.int32), labels_batch)
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    if epoch % 100 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))



In [ ]:
epoch_accuracy(tf.argmax(model(data_sets['images_test']), axis=1, output_type=tf.int32), data_sets['labels_test'])
train_loss_results.append(epoch_loss_avg.result())
print(format(epoch_accuracy.result())) #this is to see the accuracy on the test set

Here all the test accuracy will not be greater than 30%.You can clearly see that I did not create any graph.I just wrote it like a python code.This is the beauty of eager execution.This is just a basic model.We can improve the perfomance of it.